# Traffic Volume Counts Data Generator

Let's use the open source [Python SDK](https://github.com/mostly-ai/mostlyai) from [MOSTLY AI](https://mostly.ai/) to generate a synthetic dataset using [Traffic Volume Counts](https://catalog.data.gov/dataset/traffic-volume-counts) from `data.gov`.

## What is synthetic data? 

We'll use the the original traffic dataset to generate synthetic rows that preserve statistical relationships (e.g., if morning traffic is usually higher on the FDR northbound, synthetic data reflects that). 

Synthetic data is artificially generated data that mimics the patterns and structure of real data but does not contain any actual records from the original source. In our case, it is used for training machine learning and preserving privacy when real data is sensitive, incomplete, or unavailable.

## Why use synthetic data? 

There are a number of reasons to use synthetic data. Not only does it help to preserve privacy when creating open source models and datasets, but actual traffic count data is not be available for all intersections or time periods. We want to have a dataset that is as representative as possible for the entire domain.

Furthermore, we want to simulate how traffic patterns might look under unusual or hypothetical conditions—like during a marathon, blackout, or natural disaster. Generating synthetic data based on a defined subset of the whole is a perfect way to amplify certain trends.

Last, but certainly not least, is simply a matter of volume. Training an AI model is more effective the more data you have. Our base dataset is only about 50,000 rows, and once we identify a specific subset of data, it will be significantly less than that. An effective AI/ML model needs a healthy amount of data to be effectively post-trained.

# Configuring the generator

The MOSTLY AI Python SDK supports a ton of parameters to customize your synthetic data generator. The generator for these purposes is pretty simple, but the entire list of parameters can be found in the [MOSTLY AI documentation](https://mostly-ai.github.io/mostlyai/syntax/).

## Parameters

We'll focus on two parameters for this exercise:

- `max_training_time`: sets a time limit for the training process of the synthetic data generation model. Once this time limit is reached, the training stops, even if the model has not fully converged or reached its optimal state. Remember, convergence refers to the point at which the model's training process stabilizes, meaning that further training does not significantly improve the model's performance or the quality of the synthetic data being generated. Controlling for max training time allows you to make your training pipeline significantly more performative, at the expense of generating less optimal data.
- `differential_privacy`: a mathematical framework used to ensure privacy when generating synthetic data. Differential privacy provides strong guarantees that the synthetic data does not reveal sensitive information about individuals in the original dataset. When generating synthetic data, differential privacy ensures that the synthetic data cannot be used to infer sensitive information about individuals in the original dataset. `max_epsilon` is the maximum allowable value for epsilon during the synthetic data generation process. `delta` is the probability that the privacy guarantee (epsilon) could be violated (so a smaller value means a lower chance). When generating synthetic data with potentially sensitive information, it's up to the data engineer to decide the best differential privacy values for their specific use case.

In [ ]:
# Building the data generation pipeline with MOSTLY AI

import pandas as pd
from mostlyai.sdk import MostlyAI

# load original data.gov data from local repository
df_original = pd.read_csv('./data/traffic-volume-counts.csv')

# instantiate SDK
mostly = MostlyAI(local=True)

# train a generator
gen = mostly.train(config={
        'name': 'Traffic Volume Counts',          # name of the generator
        'tables': [{                              # provide list of table(s)
            'name': 'train',                      # name of the generated table
            'data': df_original,                  # the original data as pd.DataFrame
            'tabular_model_configuration': {      # tabular model configuration (optional)
                'max_training_time': 3,           # cap runtime for demo; set None for max accuracy
                'differential_privacy': {         # differential privacy configuration (optional)
                    'max_epsilon': 7.0,           # - max epsilon value, used as stopping criterion
                    'delta': 1e-5,                # - delta value
                }
            },
        }]
    },
)

# Generating a report

MOSTLY AI has a really cool feature whereby it can generate a complete report for you based on the training of your generator model. This lets you work with the configuration that you've deployed to ensure that the results meet the needs of your use case. In this example, I've used the configuration above, but maybe you'd like to tweak a few parameters and create a completely different generator? Give it a try.

In [ ]:
# Generate a report about our synthetic data generator

gen.reports(display=True)

# Targeting a subset of data

As mentioned in the introduction above, one of the key advantages to using synthetic data is the ability to target and amplify certain trends in a given dataset and then generate a synthetic dataset which is both complete and representative of subset of data that your are interested in using.

Our goal with this dataset is to build a model which can very accurately predict congestion in New York City at specific points during specific times, so it makes sense to generate a synthetic dataset based on those congested periods. Synthetic data can be used to augment real data, providing more training examples for rare events (e.g., accidents) and improving model accuracy.

In [ ]:
# use probe to inspect the underlying data

df_samples = mostly.probe(gen, size=100)
df_samples

# Defining our seed data

Let's focus on traffic patterns in the Bronx during the running of [NYC Marathon](https://en.wikipedia.org/wiki/2016_New_York_City_Marathon). This will let us build an algorithm that can tell us about traffic patterns during future runnings of the NYC Marathon so we know which parts of our neighborhood to avoid! 

In order to do so, we use the `seed` parameter and define the metadata that we'd like to use as seed data. We'll define the `Date` column from our dataset to select the date of the 2016 NYC Marathon and we use `JEROME AVENUE` for the `Roadway Name` parameter to focus on a major thoroughfare in the Bronx. 

We'll allow the generator to create data for the entire day based on the underlying dataset and then isolate the specific time periods we want to look at during post-training.

In [ ]:
df_samples = mostly.probe(gen, seed=pd.DataFrame({
    'Date': ['11/06/2016'],
    'Roadway Name':'JEROME AVENUE',
}))
df_samples

# Generating the synthetic dataset

As we know, AI and ML models require *a lot* of high quality data in order to be valuable prediction vehicles. The great thing about MOSTLY AI is we can define the amount of data that we need during the generation step and since we've already defined the seed data in the previous step, we'll get a huge amount of data that is statistically representative of our target.

Generation usually doesn't take any more than a minute depending on the complexity of the underlying data (number of columns, target output, etc.).

In [ ]:
# Define our generation parameters and print result post-generation

synth = mostly.generate(gen, size=500_000)
df_synthetic = synth.data()
df_synthetic

# Publishing our dataset

Open source development ensures that everyone can make use of the same tools that we've created here to make progress in AI and ML a truly democratic and public project.

Let's publish our data to the Hugging Face Hub in order to share this incredible dataset. For our purposes, we'd like to build an algorithm that can predict traffic patterns, but someone else might have an entirely different use case!

In [ ]:
from huggingface_hub import login

login()

In [ ]:
from datasets import Dataset

# Upload data to Hugging Face Hub
traffic = Dataset.from_pandas(df_synthetic)
traffic.push_to_hub("sythentic_marathon_traffic_volume_counts")